# ATAC-Seq Analysis for Omni-ATAC-Seq
This script starts from raw reads and performs all analyses necessary, with a final output of FastQC files, trimmed and unpaired reads, Genrich peaks, and BedGraph files.

__Required software:__ trim_galore, cutadapt, FastQC, samtools, bedtools, java, igvtools, bedGraphToBigWig, Genrich

The script expects the following input data:
1. Untrimmed .fastq.gz files in the same folder as the script.
2. A .fasta file containing the contigs that will be used for the analysis.
3. A bowtie2 library made using bowtie2-build.
4. A chrom_sizes file made using samtools and cut (see section 0.5)

The script performs the following steps:
1. Trims reads using __trim_galore__, which both trims and performs FastQC analysis.

In [ ]:
#To convert this to a Python script, use:
#jupyter nbconvert --to script Omni-ATAC_all-analysis.ipynb

#To run on a SLURM scheduler or other shared computing system, you may need to load modules.

## 0. Configuration

This section loads in required Python libraries

In [4]:
#Configuration of Python script
import subprocess
import os
import os.path
import fnmatch
import pandas as pd
import numpy as np

# 0.5. Utilities

This section contains utilities for generating necessary downstream files.

__Required software:__ samtools

In [ ]:
#############################################################
### Fill in this information before running this section ####

#Set the names of your .fasta file
genome_file = 'phaw_5.0.fa'

#Decide if you want to generatea .chrom.sizes file
make_chromsizes = False

#############################################################
#############################################################

if make_chromsizes:
    genome_index = genome_file.replace('.fa', '.fa.fai')
    chromsizes = genome_file.replace('.fa', '.chrom.sizes')
    
    !samtools faidx {genome_file}
    !cut -f1,2 {genome_index} > {chromsizes}

## 1. Trim-Galore
Run this portion of the script in a folder containing the raw Omni-ATAC reads.

Omni-ATAC data was sequenced on two different lanes of Illumina NovaSeq. </br>
Reads from the first run contain the common string "L001", while reads from the second run contain the common string "L002".

__Required software:__ trim_galore, cutadapt, FastQC

In [ ]:
#############################################################
### Fill in this information before running this section ####

#Decide if you want to run this section
run_read_trimming_run1 = False
run_read_trimming_run2 = False

#Write a base pattern for your raw fastq file names
#This pattern should occur in all of your file names
base_pattern_run1 = 'S*L001_R1_001.fastq.gz'
base_pattern_run2 = 'S*L002_R1_001.fastq.gz'

#############################################################
#############################################################

#checks if you want to trim your reads
if run_read_trimming_run1 == True:

    #Scan through all of the files in the folder specified within the single quotes in the line below
    #for each file listed in a sorted list of the directory's contents
    for file in sorted(os.listdir('.')):
        #determines if the file is a .fastq.gz file
        if fnmatch.fnmatch(file, base_pattern_run1):
            #gets the name of that file
            one = file
            #simulates the name of the other read of that file
            two = file.replace('R1_001','R2_001')
            #checks if there is not an appropriate pair for the file, if so, then exits and returns an error
            if not os.path.exists(two):
                raise Exception('A matching paired file for', file, 'was not found')
            #sends progress message to stdout
            !echo 'Performing trim-galore on' $one 'and' $two
            #performs trim_galore on the pair of reads identified, removing Nextera adapters and performing FastQC
            !trim_galore --fastqc --paired --nextera --stringency 5 --gzip --retain_unpaired $one $two
    !mv *_val_* ../2021_reanalysis/
    !mv *_unpaired_* ../2021_reanalysis/

if run_read_trimming_run2 == True:
    #Scan through all of the files in the folder specified within the single quotes in the line below
    #for each file listed in a sorted list of the directory's contents
    for file in sorted(os.listdir('.')):
        #determines if the file is a .fastq.gz file
        if fnmatch.fnmatch(file, base_pattern_run2):
            #gets the name of that file
            one = file
            #simulates the name of the other read of that file
            two = file.replace('R1_001','R2_001')
            #checks if there is not an appropriate pair for the file, if so, then exits and returns an error
            if not os.path.exists(two):
                raise Exception('A matching paired file for', file, 'was not found')
            #sends progress message to stdout
            !echo 'Performing trim-galore on' $one 'and' $two
            #performs trim_galore on the pair of reads identified, removing Nextera adapters and performing FastQC
            !trim_galore --fastqc --paired --nextera --stringency 5 --gzip --retain_unpaired $one $two
    !mv *_val_* ../2021_reanalysis/
    !mv *_unpaired_* ../2021_reanalysis/

# 2. Bowtie2 Alignment and MACS2 Peak Calling

In [ ]:
#############################################################
### Fill in this information before running this section ####

#Decide if you want to run this section
run_bowtie2_align = False
run_chrM_align = False

#Set the names of your .fasta file
genome_file = 'phaw_5.0.fa'

#Container for the chromsizes variable
bt2lib = genome_file.replace('.fa', '.bt2lib')
bt2libmarker = genome_file.replace('.fa', '.bt_marker.txt')
bt2marker_exists = False
chromsizes = genome_file.replace('.fa', '.chrom.sizes')

#Specify the path of your tool locations for Picard and igvtools
picardloc = '/clusterfs/vector/home/groups/software/sl-7.x86_64/modules/picard/2.9.0/lib/picard.jar'
igvtoolsloc = '/clusterfs/vector/home/groups/software/sl-7.x86_64/modules/igvtools/2.3.98/igvtools'
bgtobwloc = '/global/scratch/dasun/bedGraphToBigWig'

#Set the desired q-score of your read alignments
qscore = 10

#############################################################
#############################################################

#checks if you want to trim your reads
if run_bowtie2_align == True:

    #check if the bt2 marker has already been flagged
    !printf 'checking if index file has already been generated for '$genome_file'\n'
    for file in sorted(os.listdir('.')):
        if fnmatch.fnmatch(file, bt2libmarker):
            bt2marker_exists = True
            !printf 'genome index file already exists for '$genome_file'\n'

    #search through current directory
    for file in sorted(os.listdir('.')):
        #if the genome file is found and there is no bt2marker_exists flag
        if fnmatch.fnmatch(file, genome_file) and bt2marker_exists == False:
            #check for files in the current directory
            !printf 'no index file exists for '$genome_file'\n'
            !printf 'generating bt2 index for '$genome_file'\n'
            bt2_output = genome_file.replace('.fa', '_bowtie2-build-output.txt')
            !bowtie2-build -f --threads 12 $genome_file $bt2lib > $bt2_output
        
            #generate chrom.sizes file
            fainame = genome_file + '.fai'
            !samtools faidx $genome_file
            !cut -f1,2 $fainame > $chromsizes

            #make a marker file to indicate that the bowtie2 library has already been built
            !touch $bt2libmarker

    #check for files in the current directory
    for file in sorted(os.listdir('.')):
    
        #initialize and/or blank variables
        one = ''
        two = ''
        unone = ''
        untwo = ''
        prefix = ''
    
        #determine the file name and its partners, if it is R1 trimmed read
        if fnmatch.fnmatch(file,'S*L00*val_1*.fq.gz'):
            one = file
            two = file.replace('R1_001_val_1','R2_001_val_2')
            unone = one.replace('val','unpaired')
            untwo = two.replace('val','unpaired')
            !printf 'starting analysis on '$one' '$two' '$unone' '$untwo'\n'
        else: continue
    
        #determine the prefix of the file and the run
        if fnmatch.fnmatch(file, '*L001*'):
            prefix = file.partition("_")[0]+'_run1'
            !printf 'prefix is '$prefix'\n'
        elif fnmatch.fnmatch(file, '*L002*'):
            prefix = file.partition("_")[0]+'_run2'
            !printf 'prefix is '$prefix'\n'
        else: continue
        
        #initialize variables from dictionaries
        !printf 'starting analysis of '$prefix'\n'
    
        #name sam file
        samname = prefix + '.sam'
    
        #run bowtie2
        !printf 'running bowtie2 on '$prefix'\n'
        !bowtie2 --local --very-sensitive-local --threads 40 --time -x $bt2lib -1 $one -2 $two -U $unone,$untwo -S $samname

        #name bam files
        bampref = samname.replace('.sam','_sorted')
        bamname = samname.replace('.sam','_q' + str(qscore) + '.bam')
        !printf 'running samtools sort on '$prefix'\n'
        !samtools view -bS -q $qscore $samname | samtools sort -T $bampref -o $bamname
    
        #deduplicate bam files
        dedupname = bamname.replace('.bam','.dedup.bam')
        picardstats = bamname.replace('.bam','.txt')
        !printf 'running picardtools on '$prefix'\n'
        !java -jar $picardloc MarkDuplicates REMOVE_DUPLICATES=TRUE I=$bamname O=$dedupname M=$picardstats
    
        #shift files to match predicted Tn5 insertion sites
        shiftedname = bamname.replace('.bam','.shifted.bed')
        !printf 'shifting reads for '$prefix'\n'
        awkcommand = "\'BEGIN {OFS = \"\t\"} ; {if ($6 == \"+\") print $1, $2 + 4, $3 + 4, $4, $5, $6; else {if($2 - 5 > 0) print $1, $2 - 5, $3 - 5, $4, $5, $6}}\'"
        !bedtools bamtobed -i $dedupname | awk $awkcommand > $shiftedname

        #make bedgraph file
        bgname = bamname.replace('.bam','.bedgraph')
        !printf 'making bedgraph file for '$prefix'\n'
        !bedtools genomecov -i $shiftedname -g $chromsizes -bg > $bgname

        #make indexed file
        !printf 'making index file of '$prefix'\n'
        !$igvtoolsloc index $dedupname
    
        #convert bedgraph to bigwig
        !printf 'converting '$bgname' to bigwig'
        bwname = bgname.replace('.bedgraph', '.bw')
        !$bgtobwloc $bgname $chromsizes $bwname
        !printf '*** analysis of '$prefix' completed ***\n \n'

        #checks if you want to run chrMalign
        if run_chrM_align == True:
            chrMsamname = prefix+'_chrMalign.sam'
            chrMoutputfile = prefix+'_chrMalign.txt'
            !printf 'analyzing mtDNA content of '$prefix'\n'
            !bowtie2 --local --very-sensitive-local --threads 40 --time -x par.haw.chrM.bt2 -1 $one -2 $two -U $unone,$untwo -S $chrMsamname >>$chrMoutputfile 2>&1
            !printf '*** analysis of '$prefix' mtDNA contamination completed! yay! ***\n'

# 3. Run Genrich

In [ ]:
#############################################################
### Fill in this information before running this section ####

#Decide what steps you want to do
run_merge_bam = False
run_Genrich = False
run_Genrich_ATACmodeonly = False

#The settings used for the final analysis are included in this run of Genrich
run_Genrich_ATACrerun = False

#Write a dictionary with prefixes and conditions
base_pattern_2 = 'S*_*.bam'

#Specify location of Genrich
genrichloc = '/global/scratch/dasun/ATAC-Seq_phaw5.0/Genrich/Genrich'

#############################################################
#############################################################

#checks if you want to merge the bam files
if run_merge_bam == True:
    
    #makes a temporary dictionary to hold the files
    temp_dict = {}
    
    #Goes through all of the files and groups them into temp_dict
    for file in sorted(os.listdir('.')):
        if fnmatch.fnmatch(file, base_pattern_2) and 'dedup' not in file.split('.'):
            prefix = file.split('_')[0]
            if prefix not in temp_dict:
                temp_dict[prefix] = [file]
                !printf 'placed '$file' in '$prefix' bin in temp_dict\n'
            else:
                temp_dict[prefix] += [file]
                !printf 'placed '$file' in '$prefix' bin in temp_dict\n'
    
    !printf 'temp_dict contains '$temp_dict'\n'
    
    #For each condition (replicate)
    for pref in temp_dict:
        !printf 'checking runs for'$pref'\n'
        if len(temp_dict[pref]) > 1:
            !printf 'multiple runs detected for '$pref'\n'
            genrich_bam = temp_dict[pref][0].replace('.bam', '.Genrich.bam').replace('run1', 'bothruns')
            genrich_sorted = genrich_bam.replace('.Genrich', '.Genrich_sorted')
            file1 = temp_dict[pref][0]
            file2 = temp_dict[pref][1]
            !printf 'now merging '$file1' '$file2' into '$genrich_bam'\n'
            !samtools merge $genrich_bam $file1 $file2
            !printf 'now sorting '$genrich_bam' into '$genrich_sorted'\n'
            !samtools sort -n -o $genrich_sorted $genrich_bam
        else:
            !printf 'only one run detected for '$pref'\n'
            genrich_bam = temp_dict[pref][0].replace('.bam', '.Genrich.bam')
            genrich_sorted = genrich_bam.replace('.Genrich', '.Genrich_sorted')
            file1 = temp_dict[pref][0]
            !printf 'now sorting '$file1' into '$genrich_sorted'\n'
            !samtools sort -n -o $genrich_sorted $file1

#checks if you want to run Genrich
if run_Genrich == True:
    for file in sorted(os.listdir('.')):
        if 'Genrich_sorted' in file.split('.') and 'A' in list(str(file)):
            other_rep = file.replace('A', 'B')
            !printf 'now analyzing '$file' and '$other_rep'\n'
            narrowPeak = file.replace('.bam', '.narrowPeak')
            pq = file.replace('.bam', '.pq.bedgraph')
            pileup = file.replace('.bam', '.p.pileup.bedgraph')
            atac_mode = file.replace('.bam', '.ATAC.narrowPeak')
            !printf 'now performing Genrich on '$file' and '$other_rep', output to '$narrowPeak'\n'
            !$genrichloc -t $file,$other_rep -o $narrowPeak -f $pq -k $pileup -rv
            !printf 'now performing Genrich ATAC-mode on '$file' and '$other_rep', output to '$atac_mode'\n'
            !$genrichloc -t $file,$other_rep -o $atac_mode -rv

#running Genrich in ATAC mode only
if run_Genrich_ATACmodeonly == True:
    for file in sorted(os.listdir('.')):
        if 'Genrich_sorted' in file.split('.') and not 'sam' in file.split('.') and 'A' in list(str(file)):
            other_rep = file.replace('A', 'B')
            !printf 'now analyzing '$file' and '$other_rep'\n'
            pq = file.replace('.bam', '.ATAC.pq.bedgraph')
            pileup = file.replace('.bam', '.ATAC.p.pileup.bedgraph')
            atac_mode = file.replace('.bam', '.ATAC.narrowPeak')
            !printf 'now performing Genrich ATAC-mode on '$file' and '$other_rep', output to '$atac_mode'\n'
            !$genrichloc -t $file,$other_rep -o $atac_mode -f $pq -k $pileup -rv
            
#running Genrich in ATAC mode only
if run_Genrich_ATACrerun == True:
    for file in sorted(os.listdir('.')):
        if '.Genrich_sorted.bam' in file.split('_q10') and 'A' in list(str(file)):
            other_rep = file.replace('A_', 'B_')
            !printf 'now analyzing '$file' and '$other_rep'\n'
            pq = file.replace('.bam', '.ATAC.q005.bedgraph')
            pileup = file.replace('.bam', '.ATAC.q005.pileup.bedgraph')
            atac_mode = file.replace('.bam', '.ATAC.q005.narrowPeak')
            !printf 'now performing Genrich ATAC-mode on '$file' and '$other_rep', output to '$atac_mode'\n'
            !$genrichloc -t $file,$other_rep -o $atac_mode -f $pq -k $pileup -rvq 0.05

# 4. Process Genrich Output Files

Genrich generates a single bedgraph-like file that contains the bedgraph pileups from each replicate, concatenated. </br>
The bash script below splits the bedgraph-like file into two files per stage, one for each replicate.

#### Run this bash script:
`for f in S*.q005.pileup.bedgraph* ; do \
    printf 'splitting '$f'\n'`
    awk '/experimental file/{filename=NR"__'$f'"}; {print >filename}' $f
done`

In [ ]:
#############################################################
### Fill in this information before running this section ####

#Decide what steps you want to do
run_rename_Genrich = False
run_rename_Genrich_again = False
run_make_bigwig = False

#Write a dictionary with prefixes and conditions
base_pattern_3 = 'S*.ATAC.q005.pileup.bedgraph'

#############################################################
#############################################################

if run_rename_Genrich == True:
    #split and rename the file matching the base_pattern_3
    sample_files = {}
    
    for file in sorted(os.listdir('.')):
        if 'line' in file.split('_'):
            stage = file.split('_')[2]
            if stage in sample_files:
                sample_files[stage] += [file]
                !printf 'placed '$file' in '$stage' bin in temp_dict\n'
            elif stage not in sample_files:
                sample_files[stage] = [file]
                !printf 'placed '$file' in '$stage' bin in temp_dict\n'
    
    for stage in sample_files:
        A_sample = 'a'
        B_sample = 'b'
        if int(sample_files[stage][0].split('_')[0]) < int(sample_files[stage][1].split('_')[0]):
            A_sample = sample_files[stage][0]
            B_sample = sample_files[stage][1]
            !printf 'A replicate is '$A_sample' and B replicate is '$B_sample'\n'
        elif int(sample_files[stage][1].split('_')[0]) < int(sample_files[stage][0].split('_')[0]):
            A_sample = sample_files[stage][1]
            B_sample = sample_files[stage][0]
            !printf 'A replicate is '$A_sample' and B replicate is '$B_sample'\n'
        if A_sample == 'a' or B_sample == 'b':
            raise Exception('Parsing Error!')
        new_A1 = A_sample.replace('.q005.pileup.bedgraph', '.q005.pileup.split.bedgraph')
        new_A = new_A1.lstrip('_line_')
        new_B = new_A.replace('A_bothruns', 'B_bothruns')
        !printf 'renaming '$A_sample' to '$new_A'\n'
        !mv -n $A_sample $new_A
        !printf 'renaming '$B_sample' to '$new_B'\n'
        !mv -n $B_sample $new_B

if run_rename_Genrich_again == True:
    for file in sorted(os.listdir('.')):
        if 'line' in file.split('_'):
            newfilename = file.replace('q005.pileup.bedgraph', '.q005.pileup.split.bedgraph')
            newfilename2 = newfilename.lstrip('_line_')
            !mv -n $file $newfilename2

if run_make_bigwig == True:
    for file in sorted(os.listdir('.')):
        if 'split' in file.split('.'):
            newfile = file.replace('split', 'headless')
            !printf 'removingheader of '$file' and making '$newfile'\n'
            !sed '1,2d' $file > $newfile
            noctrlfile = newfile.replace('headless', 'noctrl')
            !printf 'REMOVING ctrl column from '$newfile' to '$noctrlfile'\n'
            !cut -f1,2,3,4 $newfile > $noctrlfile
            bigwigfile = noctrlfile.replace('.bedgraph', '.bw')
            !printf 'STARTING converting '$noctrlfile' to '$bigwigfile'\n'
            !./bedGraphToBigWig $noctrlfile phaw_5.0.chrom.sizes $bigwigfile
            !printf 'FINISHED converting '$noctrlfile' to '$bigwigfile'\n'

# 5. Process Genrich Output Files

This section generates a merged peak file across all timepoints by using bedtools merge. </br>
The version of the script below also generates a list of which peak numbers from each stage-specific peak file were merged into the merged peaks.

In [15]:
#############################################################
### Fill in this information before running this section ####

#Decide what steps you want to do
run_narrowPeak_name = False
run_narrowPeak_merge = False

#Write a dictionary with prefixes and conditions
narrowPeakdir = 'Genrich_sorted_bothruns/'
narrowPeakpattern = 'A_bothruns_q10.Genrich_sorted.ATAC.q005.narrowPeak'
namedPeakpattern = 'A_bothruns_q10.Genrich_sorted.ATAC.q005.named.narrowPeak'
catname = 'Scat_bothruns_q10.Genrich_sorted.ATAC.q005.named.narrowPeak'
sortedcatname = 'Scat_bothruns_q10.Genrich_sorted.ATAC.q005.sorted.narrowPeak'
mergedname = 'Sall_bothruns_q10.Genrich_sorted.ATAC.q005.merged.narrowPeak'
bampattern = 'AB_bothruns_q10.Genrich_resorted.bam'

#############################################################
#############################################################
stages = ['S13', 'S14', 'S15', 'S17', 'S18', 'S19', 'S19plus', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27']

#names peaks from each peakfile based on stage
if run_narrowPeak_name == True:
    
    #generate list of file names based on pattern and stage and directory
    narrowPeak_file_list = [narrowPeakdir + i + narrowPeakpattern for i in stages]
    
    for file in narrowPeak_file_list:
        #extract prefix from narrowPeak file name
        prefix = file.replace(narrowPeakdir, '').replace(narrowPeakpattern, '')
        
        #load in narrowPeak file
        narrowPeakdf = pd.DataFrame(pd.read_csv(file, sep = '\t', header = None))
        
        #change peak name to reflect developmental stage
        narrowPeakdf[3] = narrowPeakdf[3].str.split('_', expand = True)[1]
        narrowPeakdf[3] = prefix + '_' + narrowPeakdf[3].astype('str')
        
        #save to a new file
        newfile = file.replace('.narrowPeak', '.named.narrowPeak')
        narrowPeakdf.to_csv(newfile, sep = '\t', index = None, header = None)

if run_narrowPeak_merge == True:
    #generate list of file names based on pattern and stage and directory
    narrowPeak_file_list = [narrowPeakdir + i + namedPeakpattern for i in stages]
    
    #concatenate into string and report string
    narrowPeak_files = ' '.join(narrowPeak_file_list)
    !printf 'joining narrowPeak files: '{narrowPeak_files}'\n'
    
    #merge all into a new file with particular specifications
    #note that this new file will be located in the parent directory of all the others
    #standard narrowPeak format has these headers (using 1-index)
    #1: chrom, 2: chromStart, 3: chromEnd, 4: name, 
    #5: score, 6: strand, 7: signalValue, 8: pValue, 9: qValue, 10: peak
    #settings:
    #-count all names
    #-collapse peak names into list
    #-get mean score
    #-get mean signalValue
    #-get min q value
    #-get max q value
    !cat {narrowPeak_files} > {catname}
    !/usr/local/bin/bedtools sort -i {catname} > {sortedcatname}
    !/usr/local/bin/bedtools merge -c 4,4,5,7,9,9 -o count,collapse,mean,mean,min,max -i {sortedcatname} > {mergedname}
    
    mergeddf = pd.DataFrame(pd.read_csv(mergedname, sep = '\t', header = None))
    display(mergeddf)

joining narrowPeak files: Genrich_sorted_bothruns/S13A_bothruns_q10.Genrich_sorted.ATAC.q005.named.narrowPeak

,0,1,2,3,4,5,6,7,8
0,phaw_50.000003,4302,5642,15,"S27_0,S15_0,S23_0,S19_0,S17_0,S14_0,S25_0,S18_...",1000.000000,2054.581486,5.871578,7.869445
1,phaw_50.000009,11628,11888,1,S26_1,1000.000000,269.657501,2.950491,2.950491
2,phaw_50.000014,206406,207519,15,"S17_1,S13_1,S19plus_1,S25_1,S15_1,S20_1,S18_1,...",1000.000000,1376.176314,4.676017,7.024645
3,phaw_50.000014,238984,239410,3,"S27_2,S20_2,S21_2",1000.000000,403.039642,3.178571,4.100616
4,phaw_50.000014,239711,240244,12,"S19plus_2,S19_2,S25_2,S23_2,S24_2,S27_3,S17_2,...",944.500000,766.406303,2.249521,5.012049
...,...,...,...,...,...,...,...,...,...
190073,phaw_50.283875b,1535466,1535680,2,"S27_100575,S26_103598",1000.000000,223.467064,3.383217,3.640883
190074,phaw_50.283875b,1546841,1547714,13,"S23_108489,S25_100504,S24_106236,S20_97667,S27...",926.461538,1193.281705,2.655820,8.145149
190075,phaw_50.283875b,1548468,1548807,1,S21_114857,1000.000000,384.672821,3.372366,3.372366
190076,phaw_50.283876,1905,3313,15,"S15_63848,S17_73305,S13_65799,S19_84476,S22_99...",1000.000000,3321.552816,6.172129,7.825217
